In [43]:
import numpy as np
import re
from sklearn.pipeline import Pipeline
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import spacy
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import csv


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vicmon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/vicmon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/vicmon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [52]:
train_df = pd.read_csv(('train.csv'))
test_df = pd.read_csv(('test.csv'))

In [88]:
def data_preprocessing(sentences, vectorizer=None):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    processed_reviews = []
    
    for sentence in sentences:
        sentence = re.sub(r'[^a-zA-Z\s]', '', sentence)
        sentence = sentence.lower()
        tokens = word_tokenize(sentence)
        filtered_tokens = [word for word in tokens if word not in stop_words]
        lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
        processed_review = ' '.join(lemmatized_tokens)
        processed_reviews.append(processed_review)
    
    if vectorizer is None:
        vectorizer = CountVectorizer()
        X = vectorizer.fit_transform(processed_reviews)
    else:
        X = vectorizer.transform(processed_reviews)
    
    return X, vectorizer

In [89]:
reviews = train_df.review
labels = train_df.category
X,vectorizer = data_preprocessing(reviews)

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)


model = ComplementNB()
#model = MultinomialNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy for task1:", accuracy)


Accuracy for task1: 0.9084507042253521


In [90]:
review = test_df.review
X_test, _ = data_preprocessing(review, vectorizer)
X = X_test
prediction = model.predict(X)
results_df = pd.DataFrame({'ID': test_df['ID'],'Category': prediction})

# Write the DataFrame to a new CSV file
results_df.to_csv('predictions.csv', index=False)

In [103]:
# reviews = train_df.review
# name = train_df.name
labels = train_df.category
combine =  train_df['review'] +train_df['name'] + train_df['mean_checkin_time'].astype('str')
X,vectorizer = data_preprocessing(combine )

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)


model = ComplementNB()

#model = MultinomialNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy for task2:", accuracy)


Accuracy for task2: 0.9225352112676056


In [106]:
combine =  test_df['review'] + test_df['name']+test_df['mean_checkin_time'].astype('str')
X_test, _ = data_preprocessing(review, vectorizer)
X = X_test
prediction = model.predict(X)
results_df = pd.DataFrame({'ID': test_df['ID'],'Category': prediction})

# Write the DataFrame to a new CSV file
results_df.to_csv('predictions1.csv', index=False)